In [1]:
# API KEY Loading
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH02-Runnable")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH02-Runnable


# 프롬프트에 데이터 전달 방법

- `RunnablePassthrough` : 입력을 변경하지 않고 전달 
- `RunnablePassthrough.assign()` : 입력 받은 값을 assign 함수에 추가하여 전달
- `RunnableParallel` : 여러개 chain 묶어서 호출 
- `RunnableLambda` : 사용자 함수 맵핑

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [4]:
# Prompt 생성
prompt = PromptTemplate.from_template("{num}의 2배는?")

# model
model = ChatOpenAI(temperature=0)

# Chian
chain = prompt | model

In [ ]:
chain.invoke({"num": 20})

AIMessage(content='40입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 15, 'total_tokens': 19, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-487a9ba4-dfba-4305-a541-a9777a1ff928-0', usage_metadata={'input_tokens': 15, 'output_tokens': 4, 'total_tokens': 19, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

# RunnablePassthrough

In [7]:
from langchain_core.runnables import RunnablePassthrough

In [8]:
RunnablePassthrough().invoke({"num": 10})

{'num': 10}

In [9]:
chain_runnable = {"num": RunnablePassthrough()} | prompt | ChatOpenAI()
chain_runnable.invoke(10)

AIMessage(content='10의 2배는 20입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 15, 'total_tokens': 27, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-bc5289d0-7671-4f9d-b94b-1306de0f99c5-0', usage_metadata={'input_tokens': 15, 'output_tokens': 12, 'total_tokens': 27, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

# RunnablePassthrough.assign()

In [11]:
# num : 입력키
# new_num : 할당키
(RunnablePassthrough.assign(new_num=lambda x: x["num"] * 3)).invoke({"num": 11})

{'num': 11, 'new_num': 33}

# RunnableParallel

In [12]:
from langchain_core.runnables import RunnableParallel

In [13]:
## 앞선 실습에서....

chain1 = (
    PromptTemplate.from_template("{country}의 언어는 무엇입니까?")
    | model
    | StrOutputParser()
)

chain2 = (
    PromptTemplate.from_template("{country}의 수도는 어디입니까?")
    | model
    | StrOutputParser()
)

combine_chains = RunnableParallel(language=chain1, capital=chain2)
combine_chains.invoke({"country": "대한민국"})

{'language': '대한민국의 언어는 한국어입니다.', 'capital': '대한민국의 수도는 서울입니다.'}

In [ ]:
# RunnablePassthrough 적용해 보면...

chain1 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country}의 언어는 무엇입니까?")
    | ChatOpenAI()
)

chain2 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country}의 수도는 어디입니까?")
    | ChatOpenAI()
)

combine_chains = RunnableParallel(language=chain1, capital=chain2)
combine_chains.invoke({"country": "대한민국"})

{'language': AIMessage(content='한국어입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 32, 'total_tokens': 39, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ad6b1f6d-5a84-4688-84cc-7656967ece06-0', usage_metadata={'input_tokens': 32, 'output_tokens': 7, 'total_tokens': 39, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 'capital': AIMessage(content='서울입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 30, 'total_tokens': 36, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_to

# RunnableLambda

사용자 정의 함수 맵핑

In [19]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [ ]:
from datetime import datetime


def get_today(x):
    return datetime.today().strftime("%b-%d")

In [18]:
get_today(None)

'Jan-20'

In [ ]:
prompt = PromptTemplate.from_template(
    "{today}가 생일은 유명인 {n}명을 나열해 주세요. 생년월일도 함께 알려주세요."
)

llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

chain = (
    {"today": RunnableLambda(get_today), "n": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [21]:
print(chain.invoke(5))

1월 20일에 태어난 유명인 중 다섯 명을 소개하겠습니다.

1. **데이비드 린치 (David Lynch)** - 1946년 1월 20일 출생. 미국의 영화 감독, 시나리오 작가, 프로듀서로, 독특한 스타일과 초현실적인 작품으로 유명합니다.

2. **에반 피터스 (Evan Peters)** - 1987년 1월 20일 출생. 미국의 배우로, TV 시리즈 "아메리칸 호러 스토리"와 영화 "엑스맨" 시리즈에서의 역할로 잘 알려져 있습니다.

3. **게리 바를로우 (Gary Barlow)** - 1971년 1월 20일 출생. 영국의 가수, 작곡가, 음악 프로듀서로, 보이 밴드 테이크 댓(Take That)의 멤버로 유명합니다.

4. **폴 스탠리 (Paul Stanley)** - 1952년 1월 20일 출생. 미국의 음악가로, 록 밴드 키스(KISS)의 리드 보컬리스트이자 리듬 기타리스트로 잘 알려져 있습니다.

5. **페데리코 펠리니 (Federico Fellini)** - 1920년 1월 20일 출생. 이탈리아의 영화 감독으로, "달콤한 인생"과 "8½" 같은 고전 영화로 유명합니다.

이들은 각기 다른 분야에서 큰 영향을 미친 인물들입니다.


# itemgetter

특정 키 추출

In [22]:
from operator import itemgetter

In [23]:
# 문장의 길이 반환 함수
def length_function(text):
    return len(text)


# 두 문장의 길이를 곱한 값을 반환하는 함수
def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


# _multiple_length_function 함수를 사용하여 두 문장의 길이를 곱한 값을 반환하는 함수
def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])

In [27]:
prompt = PromptTemplate.from_template("{a}+{b}는 무엇인가요?")
model = ChatOpenAI()
chain = (
    {
        "a": itemgetter("word1") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("word1"), "text2": itemgetter("word2")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)

In [30]:
chain.invoke({"word1": "hello", "word2": "skala"})

AIMessage(content='5+25는 30입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 19, 'total_tokens': 28, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2b3187b2-a4b1-4aaa-b6eb-54523e4a43ea-0', usage_metadata={'input_tokens': 19, 'output_tokens': 9, 'total_tokens': 28, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

-----
* End of Document *